In [1]:
from PCA import *
from statistical_outlier_removal import *
from noiseApplication import *
from compareModels import *




from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
##open xyzrgb_dragon.ply mesh
pcd = o3d.io.read_point_cloud("xyzrgb_dragon.ply")


In [3]:


def hausdorff_distance(pcd1, pcd2):
    points1 = np.asarray(pcd1.points)
    points2 = np.asarray(pcd2.points)

    tree1 = cKDTree(points1)
    tree2 = cKDTree(points2)

    distances1, _ = tree1.query(points2, k=1)
    distances2, _ = tree2.query(points1, k=1)

    hausdorff_dist = max(np.max(distances1), np.max(distances2))
    return hausdorff_dist

def chamfer_distance(pcd1, pcd2):
    points1 = np.asarray(pcd1.points)
    points2 = np.asarray(pcd2.points)

    tree1 = cKDTree(points1)
    tree2 = cKDTree(points2)

    distances1, _ = tree1.query(points2, k=1)
    distances2, _ = tree2.query(points1, k=1)

    chamfer_dist = np.mean(distances1) + np.mean(distances2)
    return chamfer_dist

def compare_models(pca1, pca2):
    

    comparison_metrics = {
        'hausdorff_distance': hausdorff_distance(pca1, pca2),
        'chamfer_distance': chamfer_distance(pca1, pca2)
    }
    return comparison_metrics

In [8]:
#we have 3 kinds of noise
#1. salt and pepper noise
#2. random noise
#3. Scanner noise
# we will test out each effect sepperatly, then apply our denoising methods to remove the noise
# and compare the results to see which method performs best on each type of noise

# variables
saltnoise=1
randomnoise=1
scannernoise=0.3
scannerThreshold = 1


#1. salt and pepper noise
salt_noise_mesh = salt_and_pepper_noise(pcd, saltnoise)
#2. random noise
random_noise_mesh= apply_random_noise(pcd, randomnoise)
#3. scanner noise
scanner_noise_mesh = simulate_scanning_error(pcd, distance_threshold=scannerThreshold,noise_level=scannernoise)
#combination of all 3
all_noise_mesh = apply_random_noise(scanner_noise_mesh, randomnoise)
all_noise_mesh = salt_and_pepper_noise(all_noise_mesh, saltnoise)
all_noise_mesh = simulate_scanning_error(all_noise_mesh, distance_threshold=scannerThreshold,noise_level=scannernoise)




In [9]:
#we need to evaluate the noisy mesh as well
salt_noise_metrics=compare_models(pcd, salt_noise_mesh)
random_noise_metrics=compare_models(pcd, random_noise_mesh)
scanner_noise_metrics=compare_models(pcd, scanner_noise_mesh)

all_noise_metrics=compare_models(pcd, all_noise_mesh)


#also print the default noisy mesh
print("Salt and pepper noise")
print(salt_noise_metrics)
print("Random noise")
print(random_noise_metrics)
print("Scanner noise")
print(scanner_noise_metrics)
print("All noise")
print(all_noise_metrics)

Salt and pepper noise
{'hausdorff_distance': 1.6875634910790187, 'chamfer_distance': 0.6131885283731144}
Random noise
{'hausdorff_distance': 1.6880768951360006, 'chamfer_distance': 0.6131602953628529}
Scanner noise
{'hausdorff_distance': 4.753546744926373, 'chamfer_distance': 0.02951600153796625}
All noise
{'hausdorff_distance': 5.906750758424517, 'chamfer_distance': 0.8069311760580873}


In [10]:
#now we apply the different denoising methods individually to each type of noise
#1. salt and pepper noise
salt_noise_PCA = pca_denoising(salt_noise_mesh)
salt_noise_SOR = apply_statistical_outlier_removal(salt_noise_mesh)
salt_noise_NDOR = apply_normal_deviation_removal(salt_noise_mesh)
#2. random noise
random_noise_PCA = pca_denoising(random_noise_mesh)
random_noise_SOR = apply_statistical_outlier_removal(random_noise_mesh)
random_noise_NDOR = apply_normal_deviation_removal(random_noise_mesh)
#3. scanner noise
scanner_noise_PCA = pca_denoising(scanner_noise_mesh)
scanner_noise_SOR = apply_statistical_outlier_removal(scanner_noise_mesh)
scanner_noise_NDOR = apply_normal_deviation_removal(scanner_noise_mesh)
#combination of all 3
all_noise_PCA = pca_denoising(all_noise_mesh)
all_noise_SOR = apply_statistical_outlier_removal(all_noise_mesh)
all_noise_NDOR = apply_normal_deviation_removal(all_noise_mesh)

#1 with all denoiser applied
all_noise_all_denosiers = pca_denoising(all_noise_mesh)
all_noise_all_denosiers = apply_statistical_outlier_removal(all_noise_all_denosiers)
all_noise_all_denosiers = apply_normal_deviation_removal(all_noise_all_denosiers)


Applying Statistical Outlier Removal...
Estimating normals...
Computing normal deviations...
Applying Statistical Outlier Removal...
Estimating normals...
Computing normal deviations...
Applying Statistical Outlier Removal...
Estimating normals...
Computing normal deviations...
Applying Statistical Outlier Removal...
Estimating normals...
Computing normal deviations...
Applying Statistical Outlier Removal...
Estimating normals...
Computing normal deviations...


In [12]:
#now we want to compare the models to eachother and see which denoising method performs best on each type of noise
salt_noise_PCA_metrics=compare_models(pcd, salt_noise_PCA)
salt_noise_SOR_metrics=compare_models(pcd, salt_noise_SOR)
salt_noise_NDOR_metrics=compare_models(pcd, salt_noise_NDOR)

random_noise_PCA_metrics=compare_models(pcd, random_noise_PCA)
random_noise_SOR_metrics=compare_models(pcd, random_noise_SOR)
random_noise_NDOR_metrics=compare_models(pcd, random_noise_NDOR)

scanner_noise_PCA_metrics=compare_models(pcd, scanner_noise_PCA)
scanner_noise_SOR_metrics=compare_models(pcd, scanner_noise_SOR)
scanner_noise_NDOR_metrics=compare_models(pcd, scanner_noise_NDOR)

all_noise_PCA_metrics=compare_models(pcd, all_noise_PCA)
all_noise_SOR_metrics=compare_models(pcd, all_noise_SOR)
all_noise_NDOR_metrics=compare_models(pcd, all_noise_NDOR)

all_noise_all_denosiers_metrics=compare_models(pcd, all_noise_all_denosiers)




In [13]:
#now we want to plot the results
#1. salt and pepper noise
print("Salt and pepper noise")
print("PCA")
print(salt_noise_PCA_metrics)
print("SOR")
print(salt_noise_SOR_metrics)
print("NDOR")
print(salt_noise_NDOR_metrics)

#2. random noise
print("Random noise")
print("PCA")
print(random_noise_PCA_metrics)
print("SOR")
print(random_noise_SOR_metrics)
print("NDOR")
print(random_noise_NDOR_metrics)

#3. scanner noise
print("Scanner noise")
print("PCA")
print(scanner_noise_PCA_metrics)
print("SOR")
print(scanner_noise_SOR_metrics)
print("NDOR")
print(scanner_noise_NDOR_metrics)

#combination of all 3
print("All noise")
print("PCA")
print(all_noise_PCA_metrics)
print("SOR")
print(all_noise_SOR_metrics)
print("NDOR")
print(all_noise_NDOR_metrics)
print("All denosiers")
print(all_noise_all_denosiers_metrics)



Salt and pepper noise
PCA
{'hausdorff_distance': 1.6875634910790187, 'chamfer_distance': 0.7566772778391958}
SOR
{'hausdorff_distance': 28.4546063454545, 'chamfer_distance': 0.5851578257787091}
NDOR
{'hausdorff_distance': 1.6875634910790187, 'chamfer_distance': 0.639523118417348}
Random noise
PCA
{'hausdorff_distance': 1.6880768951360006, 'chamfer_distance': 0.7566237312747687}
SOR
{'hausdorff_distance': 28.438967305432016, 'chamfer_distance': 0.5851053253843647}
NDOR
{'hausdorff_distance': 1.6880768951360006, 'chamfer_distance': 0.6400940042519235}
Scanner noise
PCA
{'hausdorff_distance': 69.06686247195495, 'chamfer_distance': 23.394274464998364}
SOR
{'hausdorff_distance': 29.380674565168846, 'chamfer_distance': 0.01736558926657847}
NDOR
{'hausdorff_distance': 4.753546744926373, 'chamfer_distance': 0.16814669048762432}
All noise
PCA
{'hausdorff_distance': 5.906750758424517, 'chamfer_distance': 0.9615110945506762}
SOR
{'hausdorff_distance': 27.78820171513201, 'chamfer_distance': 0.7555

In [14]:
#also print the default noisy mesh
print("Salt and pepper noise")
print(salt_noise_metrics)
print("Random noise")
print(random_noise_metrics)
print("Scanner noise")
print(scanner_noise_metrics)
print("All noise")
print(all_noise_metrics)

Salt and pepper noise
{'hausdorff_distance': 1.6875634910790187, 'chamfer_distance': 0.6131885283731144}
Random noise
{'hausdorff_distance': 1.6880768951360006, 'chamfer_distance': 0.6131602953628529}
Scanner noise
{'hausdorff_distance': 4.753546744926373, 'chamfer_distance': 0.02951600153796625}
All noise
{'hausdorff_distance': 5.906750758424517, 'chamfer_distance': 0.8069311760580873}


In [15]:
#now we want know know which denoising method performs best on each type of noise
#difference is a list of measuremenst: [hausdorff, chamfer, surface area difference]
# comparison_metrics = {
#         'hausdorff_distance': hausdorff_distance(mesh1, mesh2),
#         'chamfer_distance': chamfer_distance(mesh1, mesh2),
#         'surface_area_difference': surface_area_difference(mesh1, mesh2),
#         'volume_overlap': volume_overlap(mesh1, mesh2)
#     }
# volume_overlap is not used in this comparison because the models are not watertight

#we want to see which denoising has the lowest combined number of these 3 metrics
#1. salt and pepper noise
salt_noises_PCA_combined = sum(salt_noise_PCA_metrics.values())
salt_noises_SOR_combined = sum(salt_noise_SOR_metrics.values())
salt_noises_NDOR_combined = sum(salt_noise_NDOR_metrics.values())

#2. random noise
random_noises_PCA_combined = sum(random_noise_PCA_metrics.values())
random_noises_SOR_combined = sum(random_noise_SOR_metrics.values())
random_noises_NDOR_combined = sum(random_noise_NDOR_metrics.values())

#3. scanner noise
scanner_noises_PCA_combined = sum(scanner_noise_PCA_metrics.values())
scanner_noises_SOR_combined = sum(scanner_noise_SOR_metrics.values())
scanner_noises_NDOR_combined = sum(scanner_noise_NDOR_metrics.values())

#combination of all 3
all_noises_PCA_combined = sum(all_noise_PCA_metrics.values())
all_noises_SOR_combined = sum(all_noise_SOR_metrics.values())
all_noises_NDOR_combined = sum(all_noise_NDOR_metrics.values())

all_noises_all_denosiers_combined = sum(all_noise_all_denosiers_metrics.values())





In [16]:
# if(salt_noise_PCA_combined<salt_noise_SOR_combined and salt_noise_PCA_combined<salt_noise_NDOR_combined):
#     print("Salt and pepper noise: PCA performs best")
# elif(salt_noise_SOR_combined<salt_noise_PCA_combined and salt_noise_SOR_combined<salt_noise_NDOR_combined):
#     print("Salt and pepper noise: SOR performs best")
# else:
#     print("Salt and pepper noise: NDOR performs best")
    
if(random_noises_PCA_combined<random_noises_SOR_combined and random_noises_PCA_combined<random_noises_NDOR_combined):
    print("Random noise: PCA performs best")
elif(random_noises_SOR_combined<random_noises_PCA_combined and random_noises_SOR_combined<random_noises_NDOR_combined):
    print("Random noise: SOR performs best")
else:
    print("Random noise: NDOR performs best")

if(scanner_noises_PCA_combined<scanner_noises_SOR_combined and scanner_noises_PCA_combined<scanner_noises_NDOR_combined):
    print("Scanner noise: PCA performs best")
elif(scanner_noises_SOR_combined<scanner_noises_PCA_combined and scanner_noises_SOR_combined<scanner_noises_NDOR_combined):
    print("Scanner noise: SOR performs best")
else:
    print("Scanner noise: NDOR performs best")

if(all_noises_PCA_combined<all_noises_SOR_combined and all_noises_PCA_combined<all_noises_NDOR_combined):
    print("All noise: PCA performs best")
elif(all_noises_SOR_combined<all_noises_PCA_combined and all_noises_SOR_combined<all_noises_NDOR_combined):
    print("All noise: SOR performs best")
else:
    print("All noise: NDOR performs best")



Random noise: NDOR performs best
Scanner noise: NDOR performs best
All noise: NDOR performs best
